# Individual Differences in Degree of Discounting

This notebook replicates the primary analyses from the publication:

> Wan, H., Myerson, J., & Green, L. (2023). Individual differences in degree of discounting: Do different procedures and measures assess the same construct?. *Behavioural Processes*, *208*, 104864. https://doi.org/10.1016/j.beproc.2023.104864

The analyses are translated from their original R implementation into a Python workflow using `pandas`, `lmfit`, and `statsmodels`.

The data can be requested from me or from my coauthors, Leonard Green and Joel Myerson.

In [ ]:
# Install necessary packages
import sys
!{sys.executable} -m pip install pandas numpy scipy lmfit statsmodels scikit-learn

In [4]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import auc
from lmfit import Model, Parameters
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd

warnings.filterwarnings('ignore')

# Set the display format for floating-point numbers to 3 decimal places
pd.options.display.float_format = '{:.3f}'.format

# --- Helper Functions ---
def fit_nls_logk(df):
    """Fits a simple hyperbola to individual Adj-Amt data to get log k."""
    def model_func(iv, k):
        return -np.log(1 + np.exp(k) * iv)
    
    from scipy.optimize import curve_fit
    popt, _ = curve_fit(model_func, df['iv'], np.log(df['value']), p0=[-4])
    return popt[0]

def score_mcq_logk(df):
    """Calculates theoretical log k for MCQ based on Kirby's scoring."""
    choices = df['value'].values
    k_values = df['iv'].values
    
    if all(choices == 1): return np.log(0.00016)
    if all(choices == 0): return np.log(0.25)
    
    n_consistent = [sum((choices == 0) & (k_values <= k_level) | (choices == 1) & (k_values >= k_level)) for k_level in k_values]
    max_consistency = np.max(n_consistent)
    indifference_ks = k_values[np.where(n_consistent == max_consistency)]
    
    # Return log of the geometric mean
    return np.log(np.exp(np.mean(np.log(indifference_ks))))

# --- Load and Process Data ---
disc_df = pd.read_csv("R Code/data/AdjAmt_MCQ.csv").iloc[:, 1:]
print("Data loaded successfully.")

# --- Individual-Level Data Processing ---
# Process Adj-Amt Data
adj_amt_processed = disc_df[
    (disc_df['procedure'] == "aa") & (disc_df['iv'] != 730)
].groupby(['id', 'procedure', 'amt']).apply(lambda g: pd.Series({
    'atheoretical': auc(g['iv'] / 180, g['value']),
    'theoretical': fit_nls_logk(g)
})).reset_index()

# Process MCQ Data
mcq_processed = disc_df[
    disc_df['procedure'] == "mcq"
].groupby(['id', 'amt']).apply(
    lambda g: pd.Series({
        'atheoretical': g['value'].sum(),
        'theoretical': score_mcq_logk(g)
    })
).reset_index()
mcq_processed['procedure'] = 'mcq'

# Combine and add provider info
provider_map = disc_df[['id', 'provider']].drop_duplicates()
behav = pd.concat([adj_amt_processed, mcq_processed]).merge(provider_map, on='id')

print("\nIndividual-level data processed.")
behav.head()

Data loaded successfully.

Individual-level data processed.


,id,procedure,amt,atheoretical,theoretical,provider
0,1,aa,1,0.702,-5.220,Prolific
1,1,aa,2,0.598,-4.875,Prolific
2,1,aa,3,0.884,-6.673,Prolific
3,2,aa,1,0.062,-1.109,Prolific
4,2,aa,2,0.121,-1.734,Prolific


## 2. Results

### 2.1 Group-Level Analyses & Amount Effects

This section replicates the group-level model fits and tests for the magnitude effect, where discounting decreases as the reward amount increases.

In [5]:
# --- Define Group-Level Model Functions ---
def hyperboloid_model(iv, k, b):
    """Hyperboloid discounting function for Adj-Amt group data."""
    return 1 / (1 + np.exp(k) * iv)**b

def logistic_growth_model(iv, x, r):
    """Logistic growth function for MCQ group data."""
    return 1 / (1 + np.exp(-(iv - x) * r))

# --- Group-Level Data Frame ---
disc_grp_df_agg = disc_df.groupby(['provider', 'procedure', 'amt', 'iv'])['value'].agg(['mean', 'median']).reset_index()
disc_grp_df_agg['iv_log'] = np.log(disc_grp_df_agg['iv'])

# --- 1. Replicate Group-Level Nonlinear Fits and R-squared ---
print("--- Group-Level Nonlinear Model Fit (R-squared) ---")

# --- Adj-Amt Procedure ---
aa_model = Model(hyperboloid_model)
r2_aa = disc_grp_df_agg[disc_grp_df_agg['procedure'] == 'aa'].groupby(['provider', 'amt']).apply(
    lambda g: aa_model.fit(g['median'], iv=g['iv'], k=-4, b=1).rsquared
).unstack(level='provider')
print("\nAdj-Amt R-squared:")
print(r2_aa)

# --- MCQ Procedure ---
mcq_model = Model(logistic_growth_model)
r2_mcq = disc_grp_df_agg[disc_grp_df_agg['procedure'] == 'mcq'].groupby(['provider', 'amt']).apply(
    lambda g: mcq_model.fit(g['mean'], iv=g['iv_log'], x=-4, r=1).rsquared
).unstack(level='provider')
print("\nMCQ R-squared:")
print(r2_mcq)


# --- 2. Replicate Amount Effect Tests ---
# --- Adj-Amt: Amount Effect ---
print("\n--- Adj-Amt: Amount Effect ---")
for provider_name, provider_df in behav[behav['procedure'] == 'aa'].groupby('provider'):
    provider_df['atheoretical_adj'] = np.clip(provider_df['atheoretical'], 1e-5, 1 - 1e-5)
    
    model = smf.glm("atheoretical_adj ~ C(amt, Treatment(reference=1))", data=provider_df,
                    family=sm.families.Binomial())
    
    fit = model.fit(cov_type='cluster', cov_kwds={'groups': provider_df['id']})
    
    contrast_matrix = np.array([
        [-1, 0, 1]
    ])
    wald_test = fit.wald_test(contrast_matrix)
    print(f"Provider {provider_name}: p-value = {wald_test.pvalue:.4f}")

# --- MCQ: Amount Effect ---
print("\n--- MCQ: Amount Effect ---")
for provider_name, provider_df in behav[behav['procedure'] == 'mcq'].groupby('provider'):
    
    provider_df['prop_delayed'] = provider_df['atheoretical'] / 9.0
    provider_df['n_trials'] = 9
    
    model = smf.glm("prop_delayed ~ C(amt, Treatment(reference=1))", data=provider_df,
                    family=sm.families.Binomial(),
                    weights=provider_df['n_trials'])
    
    fit = model.fit(cov_type='cluster', cov_kwds={'groups': provider_df['id']})

    wald_test = fit.wald_test(contrast_matrix)
    print(f"Provider {provider_name}: p-value = {wald_test.pvalue:.4f}")

--- Group-Level Nonlinear Model Fit (R-squared) ---

Adj-Amt R-squared:
provider  MTurk  Prolific
amt                      
1         0.882     0.980
2         0.996     0.982
3         0.987     0.979

MCQ R-squared:
provider  MTurk  Prolific
amt                      
1         0.991     0.991
2         0.982     0.984
3         0.965     0.996

--- Adj-Amt: Amount Effect ---
Provider MTurk: p-value = 0.0000
Provider Prolific: p-value = 0.0000

--- MCQ: Amount Effect ---
Provider MTurk: p-value = 0.0000
Provider Prolific: p-value = 0.0000


### 2.2 & 2.3 Reliability and Construct Validity Correlations

These analyses replicate the correlation tables from the paper, examining reliability within each procedure and validity between them.

In [6]:
# --- Within-Procedure, Within-Measure Correlations ---
print("--- Within-Procedure, Within-Measure Correlations ---")

# 1. Melt the DataFrame from wide to long format for this specific analysis
behav_long = behav.melt(
    id_vars=['id', 'procedure', 'provider', 'amt'],
    value_vars=['atheoretical', 'theoretical'],
    var_name='measure',
    value_name='value'
)

# 2. Pivot the long data to get amounts as columns
within_proc_pivot = behav_long.pivot_table(
    index=['id', 'procedure', 'provider', 'measure'], 
    columns='amt', 
    values='value'
).reset_index()

# 3. Calculate correlations
corr_results = within_proc_pivot.groupby(['measure', 'procedure', 'provider'])[[1, 2, 3]].corr()
print(corr_results.unstack().iloc[:, [1, 2, 5]])


# --- Within-Procedure, Between-Measure Correlations ---
print("\n--- Within-Procedure, Between-Measure Correlations ---")
between_measure_corr = behav.groupby(['provider', 'procedure', 'amt']).apply(
    lambda g: g['atheoretical'].corr(g['theoretical'])
).unstack()
print(between_measure_corr)


# --- Between-Procedure Correlations (Construct Validity) ---
print("\n--- Between-Procedure Correlations (Construct Validity) ---")
validity_data = behav[((behav['procedure'] == 'aa') & (behav['amt'] != 3)) | 
                      ((behav['procedure'] == 'mcq') & (behav['amt'] != 2))].copy()
validity_data['amt_str'] = np.where(validity_data['amt'] == 1, '$30', '$80')

validity_pivot = validity_data.pivot_table(
    index=['id', 'provider', 'amt_str'],
    columns='procedure',
    values=['atheoretical', 'theoretical']
)
validity_pivot.columns = ['_'.join(map(str, col)).strip() for col in validity_pivot.columns.values]

validity_corr = validity_pivot.groupby(['provider', 'amt_str']).apply(
    lambda g: pd.Series({
        'cor_atheoretical': g['atheoretical_aa'].corr(g['atheoretical_mcq']),
        'cor_theoretical': g['theoretical_aa'].corr(g['theoretical_mcq'])
    })
)
print(validity_corr)

--- Within-Procedure, Within-Measure Correlations ---
amt                                 1           2
amt                                 2     3     3
measure      procedure provider                  
atheoretical aa        MTurk    0.877 0.841 0.879
                       Prolific 0.841 0.737 0.870
             mcq       MTurk    0.897 0.890 0.925
                       Prolific 0.898 0.837 0.882
theoretical  aa        MTurk    0.874 0.840 0.865
                       Prolific 0.839 0.733 0.832
             mcq       MTurk    0.887 0.841 0.895
                       Prolific 0.902 0.834 0.874

--- Within-Procedure, Between-Measure Correlations ---
amt                     1      2      3
provider procedure                     
MTurk    aa        -0.973 -0.979 -0.982
         mcq       -0.967 -0.983 -0.975
Prolific aa        -0.963 -0.958 -0.956
         mcq       -0.995 -1.000 -0.995

--- Between-Procedure Correlations (Construct Validity) ---
                  cor_atheoretical  cor

### 2.4 Comparing Degrees of Discounting

An ANOVA is used to compare the `log k` values across procedures, samples, and reward amounts.

In [7]:
print("--- ANOVA: Comparing log k Across Procedures, Samples, and Amounts ---")
anova_data = behav[((behav['procedure'] == 'aa') & (behav['amt'] != 3)) | 
                   ((behav['procedure'] == 'mcq') & (behav['amt'] != 2))].copy()
anova_data['amt'] = np.where(anova_data['amt'] == 1, '30', '80')

# Fit the ANOVA model
model = smf.ols('theoretical ~ C(provider) * C(amt) * C(procedure)', data=anova_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# --- Post-Hoc Contrasts ---
print("\n--- Post-Hoc Tukey HSD for Procedure x Amount Interaction ---")
tukey_result = pairwise_tukeyhsd(
    endog=anova_data['theoretical'], 
    groups=anova_data['procedure'] + '_' + anova_data['amt'],
    alpha=0.05
)
print(tukey_result)

--- ANOVA: Comparing log k Across Procedures, Samples, and Amounts ---
                                  sum_sq       df       F  PR(>F)
C(provider)                      454.213    1.000 131.264   0.000
C(amt)                           141.047    1.000  40.761   0.000
C(procedure)                       8.528    1.000   2.465   0.117
C(provider):C(amt)                 3.742    1.000   1.081   0.299
C(provider):C(procedure)           3.996    1.000   1.155   0.283
C(amt):C(procedure)               19.759    1.000   5.710   0.017
C(provider):C(amt):C(procedure)    0.978    1.000   0.283   0.595
Residual                        5411.919 1564.000     NaN     NaN

--- Post-Hoc Tukey HSD for Procedure x Amount Interaction ---
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
 aa_30  aa_80  -0.3749 0.0339   -0.73 -0.0197   True
 aa_30 mcq_30   0.3715 0.0362  0.0164  0.7267   True
 aa_30 